In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import random
from PIL import Image
import cv2
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
#from torchvision.transforms import v2
from torchvision import transforms
import torch.nn.functional as F # activation function
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchsummary import summary

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
DEVICE

device(type='cuda')

In [3]:
data_dir = './Dataset'
print(os.listdir(data_dir)[:38])

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',

In [18]:
def image_count(data_dir):
    print(f'\nCounting Images in Category set :')
        
    # Iterate over each category (subfolder) in the split
        
    for sub_path in sorted(os.listdir(data_dir)):
        sub_cat = os.path.join(data_dir,sub_path)
        # Count the number of files, assuming they are all images
        num_images = len([name for name in os.listdir(sub_cat) if os.path.isfile(
                os.path.join(sub_cat, name))])
        print(f"    {sub_path}: {num_images} images")


In [19]:
image_count(data_dir=data_dir)


Counting Images in Category set :
    Apple___Apple_scab: 630 images
    Apple___Black_rot: 621 images
    Apple___Cedar_apple_rust: 275 images
    Apple___healthy: 1645 images
    Blueberry___healthy: 1502 images
    Cherry_(including_sour)___Powdery_mildew: 1052 images
    Cherry_(including_sour)___healthy: 854 images
    Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 513 images
    Corn_(maize)___Common_rust_: 1192 images
    Corn_(maize)___Northern_Leaf_Blight: 985 images
    Corn_(maize)___healthy: 1162 images
    Grape___Black_rot: 1180 images
    Grape___Esca_(Black_Measles): 1383 images
    Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1076 images
    Grape___healthy: 423 images
    Orange___Haunglongbing_(Citrus_greening): 5507 images
    Peach___Bacterial_spot: 2297 images
    Peach___healthy: 360 images
    Pepper,_bell___Bacterial_spot: 997 images
    Pepper,_bell___healthy: 1478 images
    Potato___Early_blight: 1000 images
    Potato___Late_blight: 1000 images
    Pot

In [4]:

# #trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])  # Normalize the data to [-1, 1]
])

test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])  # Normalize the data to [-1, 1]
])

val_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])  # Normalize the data to [-1, 1]
])



In [5]:
dataset = datasets.ImageFolder(
   root= data_dir
)
print(dataset)
print(f'Dataset class is : {dataset.classes}')

Dataset ImageFolder
    Number of datapoints: 54305
    Root location: ./Dataset
Dataset class is : ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold',

In [6]:
torch.manual_seed(123)
      
train_size  = int(0.7*len(dataset))
val_size = int(0.15*len(dataset))
test_size = (len(dataset) - train_size - val_size)

train_ds, val_ds, test_ds = random_split(dataset, [train_size,val_size, test_size])

# Now manually apply transforms by modifying the `.dataset` attribute
train_ds.dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
val_ds.dataset = datasets.ImageFolder(root=data_dir, transform=val_transform)
test_ds.dataset = datasets.ImageFolder(root=data_dir, transform=test_transform)

print(f'Train dataset : {len(train_ds)}, {type(train_ds)}')
print(f'Test dataset : {len(test_ds)}')
print(f'Validation dataset : {len(val_ds)}')

Train dataset : 38013, <class 'torch.utils.data.dataset.Subset'>
Test dataset : 8147
Validation dataset : 8145


In [23]:
train_ds.__class__

torch.utils.data.dataset.Subset

In [24]:
for idx, (fe, la) in enumerate(train_ds):
    print(f'Datapoint :{idx} :\n {fe,la}')
    if idx ==1:
        break

Datapoint :0 :
 (tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]]), 24)
Datapoint :1 :
 (tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., 

In [7]:
trainloader = DataLoader(
    train_ds,             # Dataset to load
    batch_size=64,        # Number of samples per batch
    shuffle=True,         # Shuffle the data at every epoch
    drop_last=True         
)

validloader = DataLoader(
    val_ds,
    shuffle=False,
    batch_size=64,
    drop_last=True
)

testloader = DataLoader(
    test_ds,
    batch_size=64,
    shuffle=False    # No need to shuffle test data

)

# # Function to show an image
# def imshow(img):
#     img = img / 2 + 0.5  # Unnormalize the image
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)).squeeze(), cmap='gray')
#     plt.axis('off')
#     plt.show()

# # # Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

print(images.shape)

# # # Show one augmented image
# print('Augmented Image:')
# imshow(images[0])
# print('Label:', classes[labels[0]])

# # Get some random test images (without augmentation)
# test_dataiter = iter(testloader)
# test_images, test_labels = next(test_dataiter)

# # Show one original test image
# print('Original Test Image:')
# imshow(test_images[0])
# print('Label:', classes[test_labels[0]])

# # 3. Visualizing Augmented Images

# # Function to display a batch of images
# def imshow_batch(img_batch, labels_batch, title):
#     img_batch = img_batch / 2 + 0.5  # Unnormalize
#     npimg = torchvision.utils.make_grid(img_batch, nrow=8)
#     npimg = npimg.numpy()
#     plt.figure(figsize=(12, 6))
#     plt.imshow(np.transpose(npimg, (1, 2, 0)).squeeze())
#     plt.title(title)
#     plt.axis('off')
#     plt.show()
#     # Print labels
#     print('Labels:', ' '.join(f'{classes[labels_batch[j]]}' for j in range(len(labels_batch))))

# # Display a batch of augmented images
# print('Batch of Augmented Images:')
# imshow_batch(images[:32], labels[:32], 'Augmented Training Images')

# # Display a batch of original test images
# print('Batch of Original Test Images:')
# imshow_batch(test_images[:32], test_labels[:32], 'Original Test Images')

torch.Size([64, 3, 128, 128])


In [26]:
for idx, (feature, labels) in enumerate(trainloader):
    print(f'Batch : {idx+1} : {feature, labels}')
    if idx ==1:
        break

Batch : 1 : (tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.000

In [8]:
# 4. Defining the Convolutional Neural Network with Dropout
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # Convolutional layers............................
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,padding=1 )  #size remains 128*128
        self.conv2 = nn.Conv2d(32, 64, 3, 1 )  # size becomes 126×126
        # pooling layer..................................
        self.pool = nn.MaxPool2d(2,2)  # pool_1 size 128*128   and pool_2  size 63×63
        # Dropout layer...............................
        self.dropout_conv = nn.Dropout2d(0.25)
        self.drop_fc = nn.Dropout(0.5)

        # Dummy input to calculate the flatten size
        dummy_input = torch.zeros(1, 3, 128, 128)
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        flatten_size = x.view(1, -1).size(1)

        # fully connected layer..................
        self.fc_1 = nn.Linear(flatten_size, 1500)
        self.fc_2 = nn.Linear(1500, 38) # classer = 38

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout_conv(x)
        x = x.view(x.size(0),-1) # Flatten
        x = F.relu(self.fc_1(x))
        x = self.drop_fc(x)
        x = self.fc_2(x)

        return x


In [9]:
model = CNN_model()

model = model.to(DEVICE)

# choosing loss and Optimizer function..................
loss = nn.CrossEntropyLoss()

optimizers = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

In [ ]:
# Training Parameters and Variables............
num_epochs = 15
train_losses =[]
val_losses = []
best_val_loss = float('inf')
patience = 5  # Early stopping patience
trigger_times = 0

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) # move to GPU
        optimizers.zero_grad()  # Zero the parameter gradients
        output = model(inputs)  # compute loss
        losses = loss(output, labels)  # compute loss
        losses.backward()
        optimizers.step()

        running_train_loss += losses

    train_loss = running_train_loss / len(trainloader)
    train_losses.append(train_loss)


    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in validloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)  # Move data to GPU
            outputs = model(inputs)
            losses = loss(outputs, labels)
            running_val_loss += losses.item()
    val_loss = running_val_loss / len(validloader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        # Save the model checkpoint
        torch.save(model.state_dict(), 'CNN_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break

# 8. Loading the Best Model

model.load_state_dict(torch.load('CNN_model.pth'))

Epoch [1/10], Training Loss: 1.5864, Validation Loss: 0.9013
Epoch [2/10], Training Loss: 0.9534, Validation Loss: 0.6292
Epoch [3/10], Training Loss: 0.7606, Validation Loss: 0.5321
Epoch [4/10], Training Loss: 0.6579, Validation Loss: 0.4812
Epoch [5/10], Training Loss: 0.5869, Validation Loss: 0.4256
Epoch [6/10], Training Loss: 0.5402, Validation Loss: 0.3733
Epoch [7/10], Training Loss: 0.5051, Validation Loss: 0.3438
Epoch [8/10], Training Loss: 0.4769, Validation Loss: 0.3482
Epoch [9/10], Training Loss: 0.4523, Validation Loss: 0.2989
Epoch [10/10], Training Loss: 0.4287, Validation Loss: 0.2887


<All keys matched successfully>

In [13]:
classes = os.listdir(data_dir)[:38]
classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spid

In [29]:
classes[0]

'Apple___Apple_scab'

In [48]:
# 10. Evaluating the Final Model

def evaluate_network():
    model.eval()
    correct = 0
    total = 0
    class_correct = [0] * 38
    class_total = [0] * 38
    num_classes = 38
    
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Per-class accuracy
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i].item()
                if label < num_classes:
                    class_correct[label] += c[i].item()
                    class_total[label] += 1

    for i in range(num_classes):
        if class_total[i] > 0:
            print(f'Accuracy of class {classes[i]} : {100 * class_correct[i] / class_total[i]:.2f}%')
        else:
            print(f'Accuracy of class {classes[i]} : N/A (no samples)')

    overall_accuracy = 100 * sum(class_correct) / sum(class_total)
    print(f'\nOverall Accuracy on the test set: {overall_accuracy:.2f}%')

# Call the evaluation function
evaluate_network()

Accuracy of class Apple___Apple_scab : 74.47%
Accuracy of class Apple___Black_rot : 89.13%
Accuracy of class Apple___Cedar_apple_rust : 48.72%
Accuracy of class Apple___healthy : 89.52%
Accuracy of class Blueberry___healthy : 98.24%
Accuracy of class Cherry_(including_sour)___healthy : 89.54%
Accuracy of class Cherry_(including_sour)___Powdery_mildew : 96.75%
Accuracy of class Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot : 65.75%
Accuracy of class Corn_(maize)___Common_rust_ : 99.45%
Accuracy of class Corn_(maize)___healthy : 86.18%
Accuracy of class Corn_(maize)___Northern_Leaf_Blight : 100.00%
Accuracy of class Grape___Black_rot : 88.04%
Accuracy of class Grape___Esca_(Black_Measles) : 93.81%
Accuracy of class Grape___healthy : 98.15%
Accuracy of class Grape___Leaf_blight_(Isariopsis_Leaf_Spot) : 70.77%
Accuracy of class Orange___Haunglongbing_(Citrus_greening) : 99.54%
Accuracy of class Peach___Bacterial_spot : 88.20%
Accuracy of class Peach___healthy : 90.62%
Accuracy of clas

In [14]:
import torch
from torchvision import transforms
from PIL import Image


In [11]:
#print(model.parameters())

In [16]:
# from torchsummary import summary
# summary(model, (3, 128,128), 64, str(DEVICE))

In [10]:
import cv2
import matplotlib.pyplot as plt
import os

# 1. Set the image path (notice the use of 'r' to handle spaces properly)
img_path = r'./Dataset/Apple___Black_rot/0c7c7992-fb1c-444b-b226-69de186cea14___JR_FrgE.S 8627.JPG'

# 2. Check if the file exists
if os.path.isfile(img_path):
    # 3. Read and convert the image
    img = cv2.imread(img_path)
    if img is not None:
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Optional: Resize to avoid memory issues
        img = cv2.resize(img, (512, 512))

        # 4. Show the image
        plt.imshow(img)
        plt.axis('off')  # optional: remove axis ticks
        plt.title("Apple___Black_rot Image")
        plt.show()
    else:
        print("Image could not be loaded. It might be corrupted.")
else:
    print("File not found. Please check the path.")


: 

In [4]:
img.shape

(512, 512, 3)

In [15]:
model.load_state_dict(torch.load('CNN_model.pth', map_location=DEVICE))
model.to(DEVICE)

CNN_model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_conv): Dropout2d(p=0.25, inplace=False)
  (drop_fc): Dropout(p=0.5, inplace=False)
  (fc_1): Linear(in_features=61504, out_features=1500, bias=True)
  (fc_2): Linear(in_features=1500, out_features=38, bias=True)
)

In [2]:


# 1. Image path
#img_path = './Dataset/Apple___Black_rot/0c7c7992-fb1c-444b-b226-69de186cea14___JR_FrgE.S 8627.JPG'  # update with your image path
#img_path = './Dataset/Blueberry___healthy/0a818f22-929b-4ef8-bcdb-ac86e909ba26___RS_HL 5438.JPG'
#img_path = './Dataset/Soybean___healthy/0a45a930-7bf1-40a5-8c36-ddf1296a65a6___RS_HL 4022.JPG'
#img_path = 'G:/Data Science/ML_and_DL_project/CNN_project/Plant_Disease_Dataset_model/test\PotatoEarlyBlight2.JPG'
img_path = 'G:/Data Science/ML_and_DL_project/CNN_project/Plant_Disease_Dataset_model/test\AppleCedarRust4.JPG'

# 2. Define the same transforms used during training
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match model input
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Optional: match training normalization
])

# 3. Load and preprocess image
image = Image.open(img_path).convert('RGB')  # ensure 3 channels
image = transform(image)
image = image.unsqueeze(0)  # Add batch dimension [1, 3, 128, 128]

# 4. Move to device

image = image.to(DEVICE)

model.eval()  # Set model to evaluation mode

# 5. Make prediction
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# 6. Output prediction
print("Predicted class name:", classes[predicted.item()])

# # 2. Check if the file exists
# if os.path.isfile(img_path):
#     # 3. Read and convert the image
#     img = cv2.imread(img_path)
#     if img is not None:
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#         # Optional: Resize to avoid memory issues
#         img = cv2.resize(img, (512, 512))

#         # 4. Show the image
#         plt.imshow(img)
#         plt.axis('off')  # optional: remove axis ticks
#         plt.title(classes[predicted.item()])
#         plt.show()
#     else:
#         print("Image could not be loaded. It might be corrupted.")
# else:
#     print("File not found. Please check the path.")

<>:6: SyntaxWarning: invalid escape sequence '\A'
<>:6: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1664\3848685094.py:6: SyntaxWarning: invalid escape sequence '\A'
  img_path = 'G:/Data Science/ML_and_DL_project/CNN_project/Plant_Disease_Dataset_model/test\AppleCedarRust4.JPG'


NameError: name 'transforms' is not defined